In [1]:
import pandas as pd
import os

class ReadDataframe:
    
    def __init__(self, df, name):
        self.df = df
        self.name = os.path.splitext(os.path.basename(name))[0]
        self.dir = os.path.dirname(name)
        self.pointsP1 = []
        self.pointsP2 = []
        self.heights = []
        self.trayPoints = []
        self.P1 = []
        self.P2 = []
        self.baseplate = 'None'
        self.sensor = 'None'
        self.pcb = 'None'

    def getComponents(self):
        if 'CuW' in self.name: self.baseplate = 'CuW'
        elif 'PCB' in self.name: self.baseplate = 'PCB'
            
        if '200' in self.name: self.sensor = '200'
        elif '300' in self.name: self.sensor = '300'
            
        if 'LD' in self.name: self.pcb = 'LD'
        elif 'HD' in self.name: self.pcb = 'HD'
        
    # Function to sort hte list by second item of tuple
    def Sort_Tuple(self, tup): 
  
        # reverse = True (Sorts in descending order for Y coordinate)
        # reverse = None (Sorts in ascending order for X coordinate)
        # key is set to sort using first or second element of 
        # sublist lambda has been used 
        tup.sort(key = lambda x: x[1], reverse=True)
        tup1, tup2 = tup[:2], tup[2:]
        tup1.sort(key = lambda x: x[0])
        tup2.sort(key = lambda x: x[0])
    
        new = [tup1[0], tup2[0], tup1[1], tup2[1]]
    
        return new 
    
    def readComponentHeights(self):
        self.getComponents()
        X = self.df.index[self.df['Unnamed: 3'] == 'Z'].tolist()
        Points = []
        for x in X:
            if 'ProtomoduleThickness' in self.df['Unnamed: 2'][x]:
                Points.append(float(self.df['Unnamed: 5'][x]))
    
        self.heights = Points

    def readComponentFiducials(self):
        X = self.df.index[self.df['Unnamed: 3'] == 'X'].tolist()
        Points = []
        for x in X:
            if 'Fiducial' in self.df['Unnamed: 2'][x]:
                Points.append((float(self.df['Unnamed: 4'][x]), float(self.df['Unnamed: 4'][x + 1])))
    
        Points = self.Sort_Tuple(Points)
        if len(Points) == 4:
            self.pointsP1 = Points
    
    def readTrayFiducials(self):
        #Get index of all points that correspond to the two reference frames
        trayIdx = self.df.index[self.df['Unnamed: 2'] == 'TrayFiducial2Fine2.X'].tolist()[0]
        P1IdxCenter = self.df.index[self.df['Unnamed: 2'] == 'P1CenterPin.X'].tolist()[0]
        P1IdxOffCenter = self.df.index[self.df['Unnamed: 2'] == 'P1OffcenterPin.X'].tolist()[0]
        P2IdxCenter = self.df.index[self.df['Unnamed: 2'] == 'P2CenterPin.X'].tolist()[0]
        P2IdxOffCenter = self.df.index[self.df['Unnamed: 2'] == 'P2OffcenterPin.X'].tolist()[0]

        #Organize such that the Center is always the first element
        #Off center is always the second element
        self.trayPoints = [(0., 0.), (float(self.df['Unnamed: 3'][trayIdx]), float(self.df['Unnamed: 3'][trayIdx + 1]))]
        self.P1 = [(float(self.df['Unnamed: 3'][P1IdxCenter]),    float(self.df['Unnamed: 3'][P1IdxCenter + 1])), 
              (float(self.df['Unnamed: 3'][P1IdxOffCenter]), float(self.df['Unnamed: 3'][P1IdxOffCenter + 1]))]
        self.P2 = [(float(self.df['Unnamed: 3'][P2IdxCenter]),    float(self.df['Unnamed: 3'][P2IdxCenter + 1])), 
              (float(self.df['Unnamed: 3'][P2IdxOffCenter]), float(self.df['Unnamed: 3'][P2IdxOffCenter + 1]))]